In [ ]:
!pip install -q -U transformers datasets bitsandbytes  trl peft  huggingface_hub

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
from transformers import AutoTokenizer, BitsAndBytesConfig, AutoModelForCausalLM
import torch

model_name = "mistralai/Mistral-7B-v0.3"

config_4bit = BitsAndBytesConfig(load_in_4bit=True)

model_4bit = AutoModelForCausalLM.from_pretrained(

                                                  model_name,
                                                  quantization_config=config_4bit,
                                                  device_map="auto",
                                                  trust_remote_code=True
)

tokenizer =AutoTokenizer.from_pretrained(model_name,padding_side="left",trust_remote_code=True)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/601 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/137k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/587k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [ ]:
from datasets import load_dataset

dataset = load_dataset("MattCoddity/dockerNLcommands")
dataset

DatasetDict({
    train: Dataset({
        features: ['input', 'output', 'instruction'],
        num_rows: 2415
    })
})

In [ ]:
tokenizer.all_special_tokens

['<s>', '</s>', '<unk>']

In [ ]:
tokens = ['<system>', '<user>', '<assistant>']

for token in tokens:
    token_id = tokenizer.encode(token,add_special_tokens=False)
    print(token,token_id)

<system> [1291, 7342, 29535]
<user> [1291, 2606, 29535]
<assistant> [1291, 1257, 11911, 29535]


In [ ]:
tokenizer.pad_token

In [ ]:
special_token = {

                 'pad_token': '<PAD>',
                 'additional_special_tokens': ['<system>', '<user>', '<assistant>']

}

tokenizer.add_special_tokens(special_token)

4

In [ ]:
tokens = ['<system>', '<user>', '<assistant>','<PAD>']

for token in tokens:
    token_id = tokenizer.encode(token,add_special_tokens=False)
    print(token,token_id)

<system> [32769]
<user> [32770]
<assistant> [32771]
<PAD> [32768]


In [ ]:
len(tokenizer)

32772

In [ ]:
model_4bit.config.vocab_size

32768

In [ ]:
model_4bit.resize_token_embeddings(len(tokenizer))

The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`
The new lm_head weights will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


Embedding(32772, 4096)

In [ ]:
tokenizer.all_special_tokens

['<s>', '</s>', '<unk>', '<PAD>', '<system>', '<user>', '<assistant>']

In [ ]:
tokenizer.bos_token

'<s>'

In [ ]:
tokenizer.eos_token

'</s>'

In [ ]:
dataset['train'][0]

{'input': 'Give me a list of containers that have the Ubuntu image as their ancestor.',
 'output': "docker ps --filter 'ancestor=ubuntu'",
 'instruction': 'translate this sentence in docker command'}

In [ ]:
new_template = """<s><system>{system_prompt}</s><user>{user_prompt}</s><assistant>{model_answer}</s>"""

def format_dataset(example):

    system_prompt = example["instruction"]
    user_prompt = example["input"]
    model_answer = example["output"]

    formatted_text = new_template.format(

                                        system_prompt= system_prompt,
                                        user_prompt = user_prompt,
                                        model_answer =model_answer
    )

    return {"text": formatted_text}

format_dataset(dataset['train'][0])

{'text': "<s><system>translate this sentence in docker command</s><user>Give me a list of containers that have the Ubuntu image as their ancestor.</s><assistant>docker ps --filter 'ancestor=ubuntu'</s>"}

In [ ]:
dataset = dataset.map(format_dataset)
dataset['train']['text'][0]

Map:   0%|          | 0/2415 [00:00<?, ? examples/s]

"<s><system>translate this sentence in docker command</s><user>Give me a list of containers that have the Ubuntu image as their ancestor.</s><assistant>docker ps --filter 'ancestor=ubuntu'</s>"